This program is to analysis regression experimental results on PDSTU and PCGITA.

In [ ]:
import os
from glob import glob
import numpy as np
import numpy.matlib as matlib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
import matplotlib as mpl, matplotlib.pyplot as plt
print(mpl.__version__)
import librosa
import librosa.display
import IPython.display

In [ ]:
# filepath = '/scratch/rfyuli/MULAN-ACCENT/'
filepath = '/home/yuanyuan/Documents/MULAN-ACCENT/exp/'
filepath_data = '/home/yuanyuan/Documents/MULAN-ACCENT/data/'
datasets = ['PDSTU','PC-GITA']
tasks = ['vowel', 'read', 'spon']
feat_types = ['sas','mfcc','egemaps']
feat_types_formal = {'sas':'SAS','mfcc':'MFCC','egemaps':'eGeMAPS'}
feat_folders = {'sas':'res/cnn', 'egemaps':'opensmile', 'mfcc':'mfcc'}
targets = {'PDSTU':['overall', 'speech', 'voice'], 'PC-GITA':['UPDRS','speech']}
dimensions = {'overall':'overall severity', 'voice':'voice impairment', \
              'UPDRS': 'UPDRS', 'speech':{'PDSTU':'speech intelligibility', 'PC-GITA':'UPDRS-speech'}}
fontsize=15

In [ ]:
# plot scatter plots for predictions VS true expert ratings.
for dataset in datasets:
    dataset_targets = targets[dataset]
    for task in tasks:
        for feat in feat_types:
            feat_folder = feat_folders[feat]
            for target in dataset_targets:

                filepath_result = os.path.join(filepath, \
                                               dataset+'_'+task, \
                                               feat_folder, 'T-CONV2D_regression',\
                                              target)
                speaker_summary = pd.read_excel(os.path.join(filepath_result, 'speaker_summary.xlsx'))
                columns = speaker_summary.columns
                truth = np.array(speaker_summary[columns[-2]])
                predictions = np.array(speaker_summary[columns[-1]])
                m, b = np.polyfit(truth, predictions, 1)
                r, p = pearsonr(truth, predictions)
                if p < 0.001:
                    p_str = '$^{***}$'
                if 0.01 > p >= 0.001:
                    p_str = '$^{**}$'
                if 0.05 > p >= 0.01:
                    p_str = '$^{*}$'
                if p >= 0.05:
                        p_str = ''
                fig, ax = plt.subplots()
                ax.scatter(truth, predictions, c='b', s=50)
                ax.plot(truth, m*truth + b, 'r-', label='r=' + str(round(r, 3)) + p_str)
                
                x_min = np.floor(np.min(np.hstack((truth, predictions))))
                x_max = np.ceil(np.max(np.hstack((truth, predictions))))
                x = np.linspace(x_min, x_max, 20)
                ax.plot(x, 1.0*x + 0, color='grey', linestyle='-.', label='y=x', markersize=12)
                ax.legend(loc='upper left', fontsize=fontsize)
                if target == 'speech':
                    dimension = dimensions[target][dataset]
                else:
                    dimension = dimensions[target]
                ax.set_xlabel('true '+dimension, fontsize=fontsize)
                ax.set_ylabel('predicted '+dimension, fontsize=fontsize)
                for label in (ax.get_xticklabels() + ax.get_yticklabels()):
                    label.set_fontsize(fontsize)
#                 ax.set_xlim(min(truth)*0.6, max(truth)*1.05)
#                 ax.set_ylim(min(truth)*0.6, max(truth)*1.05)
                print(dataset+'_'+task, feat, target)
                plt.tight_layout()
                fig.savefig(os.path.join(filepath_result, dataset+'_'+task+'_'+feat+'_'+target+'_truth_predictions_scatter.pdf'), pad_inches=0)
                plt.close()

In [ ]:
# plot the MAE distribution for segment-level predictions.
for dataset in datasets:
    dataset_targets = targets[dataset]
    for task in tasks:
        for target in dataset_targets:
            print(dataset, task, target)
            mae = []
            feats = []
            for feat in feat_types:
                feat_folder = feat_folders[feat]
                filepath_result = os.path.join(filepath, \
                                               dataset+'_'+task, \
                                               feat_folder, 'T-CONV2D_regression',\
                                              target)
                model_evaluations = pd.read_excel(os.path.join(filepath_result, 'model_evaluations.xlsx'))
                mae = mae + list(model_evaluations['mae'])
                feats = feats +[feat_types_formal[feat]]*model_evaluations['mae'].shape[0]
            data = pd.DataFrame(np.arange(len(feats)*2).reshape(len(feats), 2), columns=['MAE', 'Features'])
            data['MAE'] = np.array(mae)
            data['Features'] = np.array(feats)
            fig, ax = plt.subplots(figsize=(10,8))
            ax = sns.boxplot(x='Features', y='MAE', data=data)
            ax = sns.swarmplot(x='Features', y='MAE', data=data, color='.20')
            plt.xticks(fontsize=25)
            plt.yticks(fontsize=25)
            plt.xlabel(xlabel='Features', fontsize=25)
            plt.ylabel(ylabel='MAE', fontsize=25)
            ax.artists[0].set_facecolor('blue')
            ax.artists[1].set_facecolor('red')
            ax.artists[2].set_facecolor('magenta')
            if target == 'speech':
                dimension = dimensions[target][dataset]
            else:
                dimension = dimensions[target]
            plt.savefig(os.path.join(filepath, dataset+'_'+task, dataset+'_'+task+'_boxplot_mae_perspk_segmentlevel_prediction_{}.pdf'.format(target)))

In [ ]:
# to plot scatter plots for speaker 0026 and 0030 in PC-GITA with axes of predicted speech and UPDRS.
dataset = 'PC-GITA'
task = 'read'
spk_ids = ['AVPEPUDEA0026', 'AVPEPUDEA0030']
feats = ['SAS', 'MFCC', 'eGeMAPS']
filepath_feat = {}
filepath_feat['SAS'] = os.path.join(filepath, dataset+'_'+task, 'res/cnn/T-CONV2D_regression/')
filepath_feat['MFCC'] = os.path.join(filepath, dataset+'_'+task, 'mfcc/T-CONV2D_regression/')
filepath_feat['eGeMAPS'] = os.path.join(filepath, dataset+'_'+task, 'opensmile/T-CONV2D_regression/')
fig, ax = plt.subplots(figsize=(10,8))
colors = {'SAS':'b', 'MFCC':'r', 'eGeMAPS':'m'}
for feat in feats:
    df_speech = pd.read_excel(os.path.join(filepath_feat[feat], 'speech/model_predictions.xlsx'))
    predictions_speech = df_speech['predicted_speech']
    true_speech = df_speech['true_speech']
    speakers_speech = df_speech['speaker']
    df_UPDRS = pd.read_excel(os.path.join(filepath_feat[feat], 'UPDRS/model_predictions.xlsx'))
    predictions_UPDRS = df_UPDRS['predicted_UPDRS']
    true_UPDRS = df_UPDRS['true_UPDRS']
    speakers_UPDRS = df_UPDRS['speaker']

    # spk_ids = np.unique(speakers_speech)
    for i, spk in enumerate(spk_ids):
        pred_speech_spk = predictions_speech[speakers_speech==spk]
        true_speech_spk = np.unique(true_speech[speakers_speech==spk])
        pred_UPDRS_spk = predictions_UPDRS[speakers_UPDRS==spk]
        true_UPDRS_spk = np.unique(true_UPDRS[speakers_UPDRS==spk])
        if i==0:
            ax.scatter(pred_speech_spk, pred_UPDRS_spk, c=colors[feat], marker='o', label=spk[-4:]+':'+feat)
            ax.plot(true_speech_spk, true_UPDRS_spk, 'ko', markersize=24)
        else:
            ax.scatter(pred_speech_spk, pred_UPDRS_spk, c=colors[feat], marker='^', label=spk[-4:]+':'+feat)
            ax.plot(true_speech_spk, true_UPDRS_spk, 'k^', markersize=24)
        ax.legend(fontsize=20)
ax.set_xlabel('UPDRS-speech', fontsize=25)
ax.set_ylabel('UPDRS', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.tight_layout()
plt.savefig(os.path.join(filepath_feat['SAS'], 'scatter_predictions_spk0026_0030_read.pdf'), bbox_inches='tight', pad_inches=0)

In [ ]:
# to plot scatter plots for speaker LK5 and T44 in PC-GITAPDSTU with axes of predicted speech, voice and overall.
dataset = 'PDSTU'
task = 'read'
spk_ids = ['T44', 'LK5']
feats = ['SAS', 'MFCC', 'eGeMAPS']
filepath_feat = {}
filepath_feat['SAS'] = os.path.join(filepath, dataset+'_'+task, 'res/cnn/T-CONV2D_regression/')
filepath_feat['MFCC'] = os.path.join(filepath, dataset+'_'+task, 'mfcc/T-CONV2D_regression/')
filepath_feat['eGeMAPS'] = os.path.join(filepath, dataset+'_'+task, 'opensmile/T-CONV2D_regression/')
fig, ax = plt.subplots(1, 2, figsize=(20,8))
colors = {'SAS':'b', 'MFCC':'r', 'eGeMAPS':'m'}
for feat in feats:
    df_speech = pd.read_excel(os.path.join(filepath_feat[feat], 'speech/model_predictions.xlsx'))
    predictions_speech = df_speech['predicted_speech']
    true_speech = df_speech['true_speech']
    speakers_speech = df_speech['speaker']
    
    df_voice = pd.read_excel(os.path.join(filepath_feat[feat], 'voice/model_predictions.xlsx'))
    predictions_voice = df_voice['predicted_voice']
    true_voice = df_voice['true_voice']
    speakers_voice = df_voice['speaker']
    
    df_overall = pd.read_excel(os.path.join(filepath_feat[feat], 'overall/model_predictions.xlsx'))
    predictions_overall = df_overall['predicted_overall']
    true_overall = df_overall['true_overall']
    speakers_overall = df_overall['speaker']

    # spk_ids = np.unique(speakers_speech)
    for i, spk in enumerate(spk_ids):
        pred_speech_spk = predictions_speech[speakers_speech==spk]
        true_speech_spk = np.unique(true_speech[speakers_speech==spk])
        pred_voice_spk = predictions_voice[speakers_voice==spk]
        true_voice_spk = np.unique(true_voice[speakers_voice==spk])
        pred_overall_spk = predictions_overall[speakers_overall==spk]
        true_overall_spk = np.unique(true_overall[speakers_overall==spk])
        if i==0:
            ax[0].scatter(pred_overall_spk, pred_speech_spk, c=colors[feat], marker='o', label=spk+':'+feat)
            ax[0].plot(true_overall_spk, true_speech_spk, 'ko', markersize=24)
            plt.xticks(fontsize=25)
            plt.yticks(fontsize=25)
            ax[1].scatter(pred_overall_spk, pred_voice_spk, c=colors[feat], marker='o', label=spk+':'+feat)
            ax[1].plot(true_overall_spk, true_voice_spk, 'ko', markersize=24)
        else:
            ax[0].scatter(pred_overall_spk, pred_speech_spk, c=colors[feat], marker='^', label=spk+':'+feat)
            ax[0].plot(true_overall_spk, true_speech_spk, 'k^', markersize=24)
            ax[0].legend(fontsize=20)
            ax[0].set_xlabel('overall severity', fontsize=25)
            ax[0].set_ylabel('speech intelligibility', fontsize=25)
            for label in (ax[0].get_xticklabels() + ax[0].get_yticklabels()):
                label.set_fontsize(25)
            ax[1].scatter(pred_overall_spk, pred_voice_spk, c=colors[feat], marker='^', label=spk+':'+feat)
            ax[1].plot(true_overall_spk, true_voice_spk, 'k^', markersize=24)
            ax[1].legend(fontsize=20, loc='upper left')
            ax[1].set_xlabel('overall severity', fontsize=25)
            ax[1].set_ylabel('voice impairment', fontsize=25)
            for label in (ax[1].get_xticklabels() + ax[1].get_yticklabels()):
                label.set_fontsize(25)

plt.tight_layout()
plt.savefig(os.path.join(filepath_feat['SAS'], 'scatter_predictions_spkLK5_T44_read.pdf'), bbox_inches='tight', pad_inches=0)

In [ ]:
# to plot spectrum and speech attribute score for a vowel speech segment from speaker K2 in PDSTU (used for manuscript). 
dataset = 'PDSTU'
task = 'vowel'

df_K2_sas = pd.read_excel(os.path.join(filepath, dataset+'_'+task, 'res/cnn/K2_vowel_manner_place_dynamics.xlsx'))
K2_sas = df_K2_sas.values[:, 1:18]
attributes = list(df_K2_sas.columns[1:18])
K2_audio, sr = librosa.load(os.path.join(filepath_data,dataset+'_'+task, 'K2_vowel.wav'))
K2_audio_8k = librosa.resample(K2_audio, sr, 8000)
time_start = 2.1
time_end = 2.6
frame_start = round(time_start*8000)
frame_end = round(time_end*8000)
frame_start_sas = round(time_start/0.010)
frame_end_sas = round((time_end-0.025)/0.010)
fig, ax = plt.subplots(2, 1, figsize=(15, 10))
D = librosa.amplitude_to_db(np.abs(librosa.stft(K2_audio_8k[frame_start:frame_end], hop_length=128)), ref=np.max)
librosa.display.specshow(D, y_axis='linear', sr=8000, hop_length=128, x_axis='time', ax=ax[0])
ax[0].set_xlabel('Time (s)', fontsize=20)
ax[0].set_ylabel('Frequency (Hz)', fontsize=20)
ax[0].text(0.1, 3500, '/i:/', size=20, color='w')
ax[0].text(0.3, 3500, '/k/', size=20, color='w')
ax[0].text(0.4, 3500, '/a/', size=20, color='w')
ax[0].plot([0.24, 0.24], [0, 4000], '--', color='r')
ax[0].plot([0.36, 0.36], [0, 4000], '--', color='r')
for label in (ax[0].get_xticklabels() + ax[0].get_yticklabels()):
    label.set_fontsize(20)
# ax[0].set_xticklabels(np.array(ax[0].get_xticklabels)+2.1)
# fig, ax = plt.subplots()
ax[1] = librosa.display.specshow(K2_sas[frame_start_sas:frame_end_sas, :].T)
# attributes.reverse()
ax[1].set_yticks(np.arange(17)+0.5)
ax[1].set_yticklabels(attributes, fontsize=15)
ax[1].set_xticks(0.96*np.array([0,6,12,18,24,30,36,42,47]))
ax[1].set_xticklabels([0, 0.06, 0.12, 0.18, 0.24, 0.3, 0.36, 0.42, 0.48], fontsize=15)
ax[1].set_xlabel('Time (s)', fontsize=20)
ax[1].plot([0.96*23.5, 0.96*23.5], [0, 17], '--', color='r')
ax[1].plot([0.96*35.5, 0.96*35.5], [0, 17], '--', color='r')
for label in (ax[1].get_xticklabels() + ax[1].get_yticklabels()):
    label.set_fontsize(20)
plt.tight_layout()
plt.savefig('/home/yuanyuan/Documents/MULAN-ACCENT/journal_paper_202104/images/K2_vowel_iika(2.1s_2.5s).pdf', pad_inches=0)